In [203]:
import pickle

['price on its head : the helmeted hornbill has become a target for poachers due to the popularity of its ivory .',
 'it is one of the world ’s strangest birds , with a huge domed bill , a 6ft wingspan and a call that sounds like maniacal laughter as it echoes across the borneo rainforest .',
 'but the helmeted hornbill is being driven towards extinction – by a surge in the black - market demand for its ivory in china .',
 'the bird – dubbed ‘ the farmer of the rainforest ’ for its vital seed - spreading role – has become the latest victim of organised wildlife crime , according to a report to be published this week by the environmental investigation agency ( eia ) .']

In [211]:
document = ['price on its head : the helmeted hornbill has become a target for poachers due to the popularity of its ivory .',
 'it is one of the world ’s strangest birds , with a huge domed bill , a 6ft wingspan and a call that sounds like maniacal laughter as it echoes across the borneo rainforest .',
 'but the helmeted hornbill is being driven towards extinction – by a surge in the black - market demand for its ivory in china .',
 'the bird – dubbed ‘ the farmer of the rainforest ’ for its vital seed - spreading role – has become the latest victim of organised wildlife crime , according to a report to be published this week by the environmental investigation agency ( eia ) .',
 'its ivory is now worth three times more than that of elephants , and illegal poaching is spiralling out of control .',
 'hornbill expert yoki hadiprakarsa warned : ‘ if no one pays attention , this bird is going to become extinct . ’',
 'the illegal trade is being fuelled by the desires of china ’s wealthy middle classes for carved ivory items . with ‘ white ’ elephant ivory and rhino horn becoming more scarce , the eia found traffickers were dealing instead in ‘ red ’ ivory from a hornbill casque .',
 'this now sells for as much as £ 4,000 a kilo – three times the price of elephant ivory .',
 'there are no reliable estimates of the impact of poaching on populations across the hornbills ’ historic range of borneo , sumatra and the malaysian peninsula , but in one region alone , the kalimantan , an estimated 6,000 birds are killed every year .',
 "mr hadiprakarsa , director of the indonesian hornbill conservation society , said : ' a few years ago helmeted hornbills were common in the west kalimantan , but on my last visit i did n't see any and local villagers said they had n't either .",
 "' hunters are now pushing into the remotest areas of the forest . ’",
 'the illegal trade in ivory from the birds is being fuelled by the desires of china ’s wealthy middle classes for carved ivory items .',
 'chinese visitors look at souvenirs made from ivory at a market in angola . the ivory , which comes from helmeted hornbills , is three times the price of that of an elephant .',
 'poachers , who shoot then decapitate the birds , often return from expeditions with as many as 20 hornbill heads .',
 'these are then sold for up to £ 300 each to middle men or direct to smugglers exporting to hong kong and china .',
 'eia campaigner debbie banks said : ‘ what is sickening is this trade has nothing to do with tradition or medicine – it ’s purely about demand for luxury vanity products . ’']

sampled_documents = [document, document[0:4]]


In [212]:
summary = ['the helmeted hornbill of the borneo rainforest is being driven to extinction .',
 'it is due to a surge in the black - market demand for its ivory in china .',
 'the ivory is now worth three times more than that of elephants .',
 'report says the bird is the latest victim of organised wildlife crime .']

sampled_summaries = [summary, summary]

In [213]:
pickle.dump(sampled_documents, open("data/sampled_documents.pkl", "wb"))
pickle.dump(sampled_summaries, open("data/sampled_summaries.pkl", "wb"))

In [226]:
from torch.nn.utils.rnn import pad_sequence

a = torch.tensor([[1, 2, 3], [4, 5, 6]])
b = torch.tensor([[4, 5, 6], [7, 8, 9]])
c = torch.tensor([[4, 5, 6]])

pad_sequence([a, b, c], batch_first=True)

tensor([[[1, 2, 3],
         [4, 5, 6]],

        [[4, 5, 6],
         [7, 8, 9]],

        [[4, 5, 6],
         [0, 0, 0]]])

In [227]:
len(document)

16